In [ ]:
import os
import zipfile
import shutil
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, Xception
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Path to the uploaded .zip file
zip_file_path = '/content/drive/MyDrive/severity_data.zip'

# Directory where you want to extract the files
extract_dir = '/content/severity_dataset'

# Create the extraction directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extraction complete!")


Extraction complete!


In [ ]:
# Path to the uploaded .zip file
zip_file_path = '/content/drive/MyDrive/data.zip'

# Directory where you want to extract the files
extract_dir = '/content/data_set'

# Create the extraction directory if it doesn't exist
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extraction complete!")


Extraction complete!


In [ ]:
# Function to train model for each disease using MobileNet
def train_model_for_disease(disease_folder):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.15)

    train_generator = datagen.flow_from_directory(
        os.path.join('/content/severity_dataset/severity_data', disease_folder),
        target_size=(224,224),
        batch_size=5,
        class_mode='binary',  # Adjust for binary classification mild/severe
        subset='training')

    validation_generator = datagen.flow_from_directory(
        os.path.join('/content/severity_dataset/severity_data', disease_folder),
        target_size=(224,224),
        batch_size=5,
        class_mode='binary',  # Adjust for binary classification mild/severe
        subset='validation')

    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    steps_per_epoch = max(train_generator.samples // train_generator.batch_size, 1)
    validation_steps = max(validation_generator.samples // validation_generator.batch_size, 1)


    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        epochs=30,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
    )

    return model

In [ ]:
# Train models for each disease
disease_models = {}
disease_folders = os.listdir('/content/severity_dataset/severity_data')
for disease_folder in disease_folders:
    if disease_folder.startswith('.'):
        continue
    print(f"Training model for {disease_folder}...")
    model = train_model_for_disease(disease_folder)
    disease_models[disease_folder] = model


Training model for neck_blast...
Found 34 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
17225924/17225924 [==============================] - 2s 0us/step
Epoch 1/30
6/6 [==============================] - 11s 768ms/step - loss: 0.7019 - accuracy: 0.7241 - val_loss: 0.3198 - val_accuracy: 1.0000
Epoch 2/30
6/6 [==============================] - 4s 699ms/step - loss: 0.7079 - accuracy: 0.6552 - val_loss: 0.2784 - val_accuracy: 1.0000
Epoch 3/30
6/6 [==============================] - 4s 723ms/step - loss: 0.3188 - accuracy: 0.8966 - val_loss: 0.1741 - val_accuracy: 1.0000
Epoch 4/30
6/6 [==============================] - 3s 572ms/step - loss: 0.6271 - accuracy: 0.7931 - val_loss: 0.2282 - val_accuracy: 1.0000
Epoch 5/30
6/6 [==============================] - 3s 572ms/step - loss: 0.2890 - accuracy: 0.9000 - val_loss: 0.1922 - val_accuracy: 1.0000
Epoch 6/30
6/6 [==============================] - 4s 766ms/step - loss: 0.3518 - accuracy: 0.8621 - val_loss: 0.2738 - val

In [ ]:
# Function to predict severity using appropriate model
def predict_severity_for_image(img_path, disease_model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    prediction = disease_model.predict(img_array)
    return 'mild' if prediction < 0.5 else 'severe'

# Paths
larger_dataset_dir = '/content/data_set/data'
annotated_dataset_dir = '/content/annotated_dataset'

# Create annotated dataset directories
for disease_folder in disease_folders:
    if disease_folder == 'healthy':
        # Skip annotation for healthy images
        continue
    for severity_folder in ['mild', 'severe']:
        os.makedirs(os.path.join(annotated_dataset_dir, disease_folder+ '_' +  severity_folder), exist_ok=True)

# Annotate and move images
for disease_folder in os.listdir(larger_dataset_dir):
    if disease_folder == 'healthy':
        # Skip annotation for healthy images
        continue

    for img_name in os.listdir(os.path.join(larger_dataset_dir, disease_folder)):
        img_path = os.path.join(larger_dataset_dir, disease_folder, img_name)
        disease_model = disease_models[disease_folder]
        predicted_severity = predict_severity_for_image(img_path, disease_model)
        dest_path = os.path.join(annotated_dataset_dir, disease_folder + '_' + predicted_severity, img_name)
        shutil.copy(img_path, dest_path)


1/1 [==============================] - 0s 19ms/step


In [ ]:
# Paths
annotated_dataset_dir = '/content/annotated_dataset'
class_folders = [folder_name for folder_name in os.listdir(annotated_dataset_dir) if not folder_name.startswith('.') and os.path.isdir(os.path.join(annotated_dataset_dir, folder_name))]

In [ ]:
print(len(class_folders))

18


In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.15)

train_generator = datagen.flow_from_directory(
    annotated_dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    classes=class_folders)

validation_generator = datagen.flow_from_directory(
    annotated_dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    classes = class_folders)


Found 3069 images belonging to 18 classes.
Found 531 images belonging to 18 classes.


In [ ]:
# Early stopping callback
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

'''# Function to create a CNN model
def create_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(18, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model'''


"# Function to create a CNN model\ndef create_cnn_model():\n    model = Sequential([\n        Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),\n        MaxPooling2D((2, 2)),\n        Conv2D(64, (3, 3), activation='relu'),\n        MaxPooling2D((2, 2)),\n        Conv2D(128, (3, 3), activation='relu'),\n        MaxPooling2D((2, 2)),\n        Flatten(),\n        Dense(128, activation='relu'),\n        Dropout(0.5),\n        Dense(18, activation='softmax')\n    ])\n    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])\n    return model"

In [ ]:
# Function to create a transfer learning model
def create_transfer_model(base_model):
    base_model.trainable = False
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(18, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
'''# Create and train models
cnn_model = create_cnn_model()
cnn_history = cnn_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)'''

'# Create and train models\ncnn_model = create_cnn_model()\ncnn_history = cnn_model.fit(\n    train_generator,\n    epochs=10,\n    validation_data=validation_generator,\n    callbacks=[early_stopping]\n)'

In [ ]:

vgg16_model = create_transfer_model(VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)))
vgg16_history = vgg16_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


58889256/58889256 [==============================] - 4s 0us/step
Epoch 1/10
96/96 [==============================] - 112s 1s/step - loss: 2.6254 - accuracy: 0.1949 - val_loss: 2.2621 - val_accuracy: 0.3446
Epoch 2/10
96/96 [==============================] - 95s 989ms/step - loss: 2.2023 - accuracy: 0.3262 - val_loss: 1.9528 - val_accuracy: 0.4331
Epoch 3/10
96/96 [==============================] - 98s 1s/step - loss: 1.9556 - accuracy: 0.4027 - val_loss: 1.7395 - val_accuracy: 0.5254
Epoch 4/10
96/96 [==============================] - 99s 1s/step - loss: 1.7680 - accuracy: 0.4562 - val_loss: 1.5786 - val_accuracy: 0.5292
Epoch 5/10
96/96 [==============================] - 94s 984ms/step - loss: 1.6393 - accuracy: 0.4832 - val_loss: 1.4631 - val_accuracy: 0.5537
Epoch 6/10
96/96 [==============================] - 95s 985ms/step - loss: 1.5623 - accuracy: 0.5145 - val_loss: 1.3830 - val_accuracy: 0.5782
Epoch 7/10
96/96 [==============================] - 95s 989ms/step - loss: 1.4751 - a

In [ ]:

'''resnet50_model = create_transfer_model(ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)))
resnet50_history = resnet50_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)'''


"resnet50_model = create_transfer_model(ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)))\nresnet50_history = resnet50_model.fit(\n    train_generator,\n    epochs=10,\n    validation_data=validation_generator,\n    callbacks=[early_stopping]\n)"

In [ ]:
inceptionv3_model = create_transfer_model(InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3)))
inceptionv3_history = inceptionv3_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


87910968/87910968 [==============================] - 5s 0us/step
Epoch 1/10
96/96 [==============================] - 109s 1s/step - loss: 2.3052 - accuracy: 0.3043 - val_loss: 1.4928 - val_accuracy: 0.6045
Epoch 2/10
96/96 [==============================] - 97s 1s/step - loss: 1.7030 - accuracy: 0.4751 - val_loss: 1.1899 - val_accuracy: 0.6723
Epoch 3/10
96/96 [==============================] - 95s 989ms/step - loss: 1.4252 - accuracy: 0.5438 - val_loss: 1.0137 - val_accuracy: 0.7137
Epoch 4/10
96/96 [==============================] - 96s 991ms/step - loss: 1.3105 - accuracy: 0.5810 - val_loss: 0.9878 - val_accuracy: 0.7043
Epoch 5/10
96/96 [==============================] - 97s 1s/step - loss: 1.2295 - accuracy: 0.6142 - val_loss: 0.8827 - val_accuracy: 0.7401
Epoch 6/10
96/96 [==============================] - 97s 1s/step - loss: 1.2097 - accuracy: 0.6168 - val_loss: 0.8438 - val_accuracy: 0.7345
Epoch 7/10
96/96 [==============================] - 94s 977ms/step - loss: 1.1378 - accu

In [ ]:

xception_model = create_transfer_model(Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3)))
xception_history = xception_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


83683744/83683744 [==============================] - 5s 0us/step
Epoch 1/10
96/96 [==============================] - 111s 1s/step - loss: 1.9315 - accuracy: 0.4171 - val_loss: 1.2985 - val_accuracy: 0.6139
Epoch 2/10
96/96 [==============================] - 96s 1000ms/step - loss: 1.2959 - accuracy: 0.5953 - val_loss: 1.0148 - val_accuracy: 0.7043
Epoch 3/10
96/96 [==============================] - 95s 987ms/step - loss: 1.1092 - accuracy: 0.6442 - val_loss: 0.9148 - val_accuracy: 0.6911
Epoch 4/10
96/96 [==============================] - 99s 1s/step - loss: 1.0083 - accuracy: 0.6774 - val_loss: 0.7479 - val_accuracy: 0.7608
Epoch 5/10
96/96 [==============================] - 99s 1s/step - loss: 0.9116 - accuracy: 0.7061 - val_loss: 0.7288 - val_accuracy: 0.7797
Epoch 6/10
96/96 [==============================] - 98s 1s/step - loss: 0.8971 - accuracy: 0.7120 - val_loss: 0.6924 - val_accuracy: 0.7815
Epoch 7/10
96/96 [==============================] - 101s 1s/step - loss: 0.8412 - accur

In [ ]:
# Evaluate models
def evaluate_model(model):
    val_loss, val_accuracy = model.evaluate(validation_generator)
    return val_accuracy
#cnn_accuracy = evaluate_model(cnn_model)
vgg16_accuracy = evaluate_model(vgg16_model)
inceptionv3_accuracy = evaluate_model(inceptionv3_model)
xception_accuracy = evaluate_model(xception_model)


17/17 [==============================] - 14s 858ms/step - loss: 0.6216 - accuracy: 0.7928


In [ ]:
#'CNN': cnn_accuracy,
accuracies = {

    'VGG16': vgg16_accuracy,
    'InceptionV3': inceptionv3_accuracy,
    'Xception': xception_accuracy
}

best_model_name = max(accuracies, key=accuracies.get)
best_model_accuracy = accuracies[best_model_name]

print(f"Best model: {best_model_name} with accuracy: {best_model_accuracy}")

Best model: Xception with accuracy: 0.7928436994552612


In [ ]:
best_model = None
if best_model_name == 'VGG16':
    best_model = vgg16_model
elif best_model_name == 'InceptionV3':
    best_model = inceptionv3_model
elif best_model_name == 'Xception':
    best_model = xception_model
'''
if best_model_name == 'CNN':
    best_model = cnn_model
el
'''

"\nif best_model_name == 'CNN':\n    best_model = cnn_model\nel\n"

In [ ]:
# Create ensemble predictions (average method)
def ensemble_predict(image_path, models):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = [model.predict(img_array) for model in models]
    avg_prediction = np.mean(predictions, axis=0)
    return np.argmax(avg_prediction)
#models = [cnn_model, vgg16_model, resnet50_model, inceptionv3_model, xception_model]
# Example usage for ensemble prediction
image_path = '/content/data_set/data/brown_spot/brown_spot0.jpg'
models = [vgg16_model, inceptionv3_model, xception_model]
ensemble_prediction = ensemble_predict(image_path, models)
print(f"Ensemble model prediction: {ensemble_prediction}")


1/1 [==============================] - 1s 1s/step
Ensemble model prediction: 1


In [ ]:
# Load and preprocess the image
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    return img_array

# Test the best model with an image
test_image_path = '/content/data_set/data/brown_spot/brown_spot0.jpg'
img_array = load_and_preprocess_image(test_image_path)
prediction = best_model.predict(img_array)
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}
predicted_class = class_labels[np.argmax(prediction)]
print(f'Test Image: {test_image_path}, Predicted Class: {predicted_class}')

1/1 [==============================] - 0s 22ms/step
Test Image: /content/data_set/data/brown_spot/brown_spot0.jpg, Predicted Class: brown_spot_mild


In [ ]:
# Compute ensemble accuracy on validation set
correct_predictions = 0
total_predictions = 0

for i in range(len(validation_generator)):
    x, y_true = validation_generator[i]
    for j in range(len(x)):
        img = x[j]
        true_label = np.argmax(y_true[j])
        img_path = "/content/data_set/data/brown_spot/brown_spot0.jpg"
        image.save_img(img_path, img)
        predicted_label = ensemble_predict(img_path, [vgg16_model,  inceptionv3_model, xception_model])

        if predicted_label == true_label:
            correct_predictions += 1
        total_predictions += 1

ensemble_accuracy = correct_predictions / total_predictions
print(f"Ensemble model accuracy: {ensemble_accuracy}")


1/1 [==============================] - 0s 20ms/step
Ensemble model accuracy: 0.7928436911487758


In [ ]:
if ensemble_accuracy > best_model_accuracy:
    # Save the ensemble model
    with open('ensemble_model.pkl', 'wb') as file:
        pickle.dump([vgg16_model, inceptionv3_model, xception_model], file)
    print("Ensemble model saved as ensemble_model.pkl")
else:
    # Save the best individual model
    with open('best_model.pkl', 'wb') as file:
        pickle.dump(best_model, file)
    print(f"Best model saved as best_model.pkl")


Best model saved as best_model.pkl


In [ ]:
import shutil

def zip_folder(folder_path, output_path):
    """
    Zips the contents of a folder.

    Parameters:
    folder_path (str): Path to the folder to be zipped.
    output_path (str): Path for the output zipped file.
    """
    shutil.make_archive(output_path, 'zip', folder_path)

# Example usage
folder_to_zip = '/content/annotated_dataset'  # replace with your folder name
output_zip_path = '/content/annotated_dataset_zip_folder'  # replace with your desired output zip file name

zip_folder(folder_to_zip, output_zip_path)